In [8]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np
import warnings
import sklearn
warnings.filterwarnings(action='ignore') 

In [9]:
train = pd.read_csv('./data/train_all.csv')
test = pd.read_csv('./data/test_all.csv')

In [10]:
x_train = train.drop(['predicted_weight_g'], axis=1)
y_train = pd.DataFrame(train['predicted_weight_g'])
test = test.drop(['predicted_weight_g'], axis=1)

In [14]:
x_train = x_train.fillna(method='ffill')

In [15]:
new_train1, new_target1 = OriginalGenerator().generate_data_pipe(x_train, y_train, test, )
new_train2, new_target2 = GANGenerator().generate_data_pipe(x_train, y_train, test, )

Fitting CTGAN transformers for each column:   0%|          | 0/18 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

In [22]:
new_train2

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,Case
0,22,16,29.186667,78.476668,532.100000,0.442515,123.400000,1727.600000,0.000000,0.000000,4881.660000,48618.024000,4621.175000,46023.770000,9502.835000,94641.794000,2
1,22,17,29.986666,79.113333,514.450000,0.432468,123.400000,1851.000000,0.000000,0.000000,4881.660000,53499.684000,4621.175000,50644.945000,9502.835000,104144.629000,2
2,24,17,28.125000,77.546667,501.950000,0.000000,123.400000,1851.000000,0.000000,0.000000,4881.660000,53473.207200,4621.175000,50619.881000,9502.835000,104093.088200,2
3,24,16,27.600000,78.182000,501.233333,0.000000,123.400000,1727.600000,0.000000,0.000000,4881.660000,48591.547200,4621.175000,45998.706000,9502.835000,94590.253200,2
4,23,16,27.421667,78.554999,516.666667,0.000000,123.400000,1727.600000,0.000000,0.000000,4881.660000,48609.750000,4621.175000,46015.937500,9502.835000,94625.687500,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,6,5,25.642538,65.623273,544.788357,1.025920,1494.178210,15557.522849,3057.897769,199394.277260,2969.381549,736.012965,96.079057,2247.235666,14368.532402,6265.728332,5
453,5,7,21.917739,58.270802,671.957903,1.246775,1767.905991,7507.984074,2571.652184,731.628649,120.329187,32304.152041,98.602776,6048.286729,31341.759052,6732.676158,1
454,0,8,22.989488,42.801014,358.290021,0.000333,1071.404506,136.990660,11804.840806,119496.505806,1525.239602,86220.285548,28.490077,23600.760738,19109.592590,168592.540242,2
455,14,8,33.731622,58.731770,556.032070,1.378547,316.604984,17264.265897,17384.444793,95361.434240,455.407643,31210.942478,77.880536,64194.520428,645.308767,74952.740116,1


In [93]:
new_train3, new_target3 = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "AUC", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,
           gan_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(x_train, y_train,
                                          test, deep_copy=True, only_adversarial=False, use_adversarial=True)

Fitting CTGAN transformers for each column:   0%|          | 0/163 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

In [17]:
def fit_predict(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    return sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])



dataset = sklearn.datasets.load_breast_cancer()
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=25, max_depth=6)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    pd.DataFrame(dataset.data), pd.DataFrame(dataset.target, columns=["target"]), test_size=0.33, random_state=42)
print("initial metric", fit_predict(clf, X_train, y_train, X_test, y_test))

new_train1, new_target1 = OriginalGenerator().generate_data_pipe(X_train, y_train, X_test, )
print("OriginalGenerator metric", fit_predict(clf, new_train1, new_target1, X_test, y_test))

new_train1, new_target1 = GANGenerator().generate_data_pipe(X_train, y_train, X_test, )
print("GANGenerator metric", fit_predict(clf, new_train1, new_target1, X_test, y_test))

AttributeError: module 'sklearn' has no attribute 'datasets'

In [22]:
import torch

batch_size, N, K = 3, 10, 256

x = torch.rand(batch_size, N, K) # [M, N, K]
y = torch.rand(batch_size, N, K) # [M, N, K]

output1 = torch.cat([x,y], dim=1) #[M, N+N, K]
output2 = torch.cat([x,y], dim=2) #[M, N, K+K]

In [27]:
print(x.shape)
print(y.shape)
print(output1.shape)
print(output2.shape)

torch.Size([3, 10, 256])
torch.Size([3, 10, 256])
torch.Size([3, 20, 256])
torch.Size([3, 10, 512])


torch.cat의 dim=1이면 행의 개수더하기
torch.cat의 dim=2면 열의 개수더하기
